In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wd/'

Mounted at /content/drive


In [3]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [4]:
filename = 'HV_data.csv'
HV_data = pd.read_csv(path + filename)

In [5]:
set_index(HV_data)

In [6]:
tm_data = HV_data

In [7]:
HV_data = HV_data.drop(columns=['HV'])

In [8]:
HV_data['HV'] = tm_data['HV']

In [9]:
HV_data

,"('GARCH', 'normal', 0)","('GARCH', 'normal', 1)","('GARCH', 'gaussian', 0)","('GARCH', 'gaussian', 1)","('GARCH', 'ged', 0)","('GARCH', 'ged', 1)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)","('FIGARCH', 'gaussian', 0)","('FIGARCH', 'gaussian', 1)",...,unemployment_rate,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,usd_mxn,gt_data,log_returns,HV
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-02,0.039601,0.038301,0.039601,0.038301,0.040044,0.039199,0.038388,0.038388,0.038388,0.038388,...,11.596551,2.781970,307.252039,4.529620,23.415205,4.090860,26.750733,209.303607,-0.005604,0.039337
2000-02-03,0.037979,0.038355,0.037979,0.038355,0.041000,0.040608,0.038633,0.038633,0.038633,0.038633,...,11.596551,2.799436,311.491144,4.529266,23.412659,4.119569,26.561052,209.303607,-0.008693,0.039588
2000-02-04,0.036791,0.037755,0.036791,0.037755,0.038214,0.040159,0.036990,0.036990,0.036990,0.036990,...,11.419775,2.728548,306.262089,4.474254,23.415452,4.106629,26.433950,209.303607,0.000572,0.037904
2000-02-07,0.046593,0.055072,0.046593,0.055072,0.047968,0.057512,0.046474,0.046474,0.046474,0.046474,...,11.313708,2.733357,299.491542,4.460147,23.414391,4.140641,26.339020,209.303607,0.027307,0.044257
2000-02-08,0.049131,0.041984,0.049131,0.041984,0.049560,0.042487,0.046711,0.046711,0.046711,0.046711,...,11.313708,2.730104,301.761354,4.470789,23.415982,4.142338,26.135551,209.303607,0.010478,0.049006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.000120,-0.000665,0.000120,-0.000665,0.000456,-0.000424,0.000121,0.000121,0.000121,0.000121,...,0.000000,-0.002546,0.509117,-0.003041,0.005162,0.001980,-0.105925,0.404061,0.008246,0.000123
2024-08-27,-0.003889,-0.004798,-0.003889,-0.004798,-0.003539,-0.002953,-0.004368,-0.004368,-0.004368,-0.004368,...,0.000000,0.002051,-0.551543,-0.001202,-0.006788,-0.001556,-0.173948,0.404061,0.009336,0.000656
2024-08-28,0.000280,-0.001148,0.000280,-0.001148,0.000449,-0.000383,0.000483,0.000483,0.000483,0.000483,...,0.000000,0.000849,0.247487,0.000849,0.002263,0.002051,-0.219910,0.707107,0.010307,0.000087


# Split train, validation and test data

In [10]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [11]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [12]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [13]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## HV

In [14]:
daily_train, daily_val, daily_test = split_data(HV_data, 'HV')

In [15]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [16]:
HV_X_train, HV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
HV_X_val, HV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
HV_X_test, HV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [17]:
# HV_X_train = HV_X_train.reshape(HV_X_train.shape[0], win_sz, -1)
# HV_X_val = HV_X_val.reshape(HV_X_val.shape[0], win_sz, -1)
# HV_X_test = HV_X_test.reshape(HV_X_test.shape[0], win_sz, -1)

### LSTM

In [18]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │          16,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,373 (79.58 KB)

 Trainable params: 20,373 (79.58 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
lstm_history = lstm_garch_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 0.0101 - val_loss: 1.3723e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 1.6937e-04 - val_loss: 5.9467e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 1.2241e-04 - val_loss: 6.2645e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - loss: 8.9961e-05 - val_loss: 5.6252e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - loss: 9.3204e-05 - val_loss: 3.5015e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - loss: 7.9564e-05 - val_loss: 2.5326e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - loss: 7.3488e-05 - val_loss: 2.9084e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 8.0615e-05 - val_loss: 1.0491e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 6.0358e-05 - val_loss: 5.6939e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 7.7137e-05 - val_loss: 7.8371e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step

In [21]:
# Predictions
HV_test_predictions = lstm_garch_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


In [22]:
mse = mean_squared_error(HV_y_test, HV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 7.228990239524999e-06
RMSE: 0.0026886781584126054
MAE: 0.002212884378317901
MAPE: 31116.480984335034%


### GRU

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

In [ ]:
gru_model.summary()

In [ ]:
gru_history = gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - loss: 0.0533 - val_loss: 1.4514e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 2.0091e-04 - val_loss: 5.9310e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 1.2887e-04 - val_loss: 4.0802e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 9.6715e-05 - val_loss: 3.0445e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 9.9466e-05 - val_loss: 3.5547e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 7.6158e-05 - val_loss: 4.0919e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - loss: 7.2741e-05 - val_loss: 3.5202e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 6.3532e-05 - val_loss: 3.8449e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 6.9472e-05 - val_loss: 3.0267e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 6.6837e-05 - val_loss: 3.9332e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/ste

In [ ]:
# Predictions
HV_gru_predictions = gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 1.1709414715888632e-05
RMSE: 0.0034219022072362955
MAE: 0.0031337627766335778
MAPE: 44649.79379835331%


### LSTM_GRU

In [ ]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_gru_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 25, 48)              │          16,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,381 (75.71 KB)

 Trainable params: 19,381 (75.71 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_gru_history = lstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - loss: 0.0291 - val_loss: 1.1602e-04
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 5.2867e-04 - val_loss: 1.9767e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 2.5477e-04 - val_loss: 2.1394e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 1.5634e-04 - val_loss: 1.6351e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 1.3544e-04 - val_loss: 5.1577e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 1.1162e-04 - val_loss: 5.2675e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 1.0033e-04 - val_loss: 6.2387e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - loss: 8.0679e-05 - val_loss: 3.9093e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 7.5528e-05 - val_loss: 4.0896e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 7.1112e-05 - val_loss: 3.0306e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step

In [ ]:
# Predictions
HV_lstm_gru_predictions = lstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 2.4543411124304922e-06
RMSE: 0.0015666336880172378
MAE: 0.0012387070480063768
MAPE: 17230.679909540304%


### BiLSTM

In [ ]:
bilstm_model = Sequential()
bilstm_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Bidirectional(LSTM(16)))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Dense(5))

bilstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

In [ ]:
bilstm_model.summary()

In [ ]:
bilstm_history = bilstm_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

In [ ]:
# Predictions
HV_bilstm_predictions = bilstm_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_bilstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_predictions)
print(f'MAPE: {mape}%')

MSE: 8.120603418915605e-06
RMSE: 0.002849667247051067
MAE: 0.0026374769310524722
MAPE: 37347.43550935421%


### BiLSTM_GRU

In [ ]:
bilstm_gru_model = Sequential()
bilstm_gru_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(GRU(16))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(Dense(5))

bilstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
bilstm_gru_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_3 (Bidirectional)      │ (None, 25, 96)              │          32,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_4 (GRU)                          │ (None, 16)                  │           5,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 37,813 (147.71 KB)

 Trainable params: 37,813 (147.71 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bilstm_gru_history = bilstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 13s 45ms/step - loss: 0.0339 - val_loss: 9.6630e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - loss: 4.1968e-04 - val_loss: 1.5072e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 1.9548e-04 - val_loss: 5.2549e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 1.2985e-04 - val_loss: 1.0680e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - loss: 9.6946e-05 - val_loss: 2.4289e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - loss: 8.5007e-05 - val_loss: 3.6795e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - loss: 8.2091e-05 - val_loss: 2.0624e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - loss: 7.4086e-05 - val_loss: 1.6963e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - loss: 6.1681e-05 - val_loss: 5.0886e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - loss: 6.7297e-05 - val_loss: 1.3797e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32

In [ ]:
# Predictions
HV_bilstm_gru_predictions = bilstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 6.472612620134574e-07
RMSE: 0.0008045254887282673
MAE: 0.0006273253992673622
MAPE: 7969.562201704585%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/HV/'

# df_pred = pd.DataFrame(HV_test_predictions[:,0], columns=['lstm_pred'])
# filename = 'LSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_gru_predictions[:,0], columns=['gru_pred'])
# filename = 'GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_predictions[:,0], columns=['bilstm_pred'])
# filename = 'BiLSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_gru_predictions[:,0], columns=['bilstm_gru_pred'])
# filename = 'BiLSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)